In [1]:
import os,sys
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch
import shutil
import time
import datetime
import importlib
from loguru import logger

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import modules
from bivme.preprocessing.dicom.extract_cines import extract_cines
from bivme.preprocessing.dicom.select_views import select_views
from bivme.preprocessing.dicom.segment_views import segment_views
from bivme.preprocessing.dicom.correct_phase_mismatch import correct_phase_mismatch
from bivme.preprocessing.dicom.generate_contours import generate_contours
from bivme.preprocessing.dicom.export_guidepoints import export_guidepoints
from bivme.plotting.plot_guidepoints import generate_html # for plotting guidepoints

In [ ]:
# Set up logging
log_level = "DEBUG"
log_format = "<green>{time:YYYY-MM-DD HH:mm:ss.SSS zz}</green> | <level>{level: <8}</level> | <yellow>Line {line: >4} ({file}):</yellow> <b>{message}</b>"

In [4]:
# Check if GPU is available (torch)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

2025-03-19 17:05:21.923 | INFO     | __main__:<module>:3 - Using device: cuda


## This code reads in DICOM files and generates GPFiles for personalised biventricular mesh fitting.

## Step 0: Set up directories
All directories will be created for you except for the input_path. This should point to your DICOMs, separated into folders by case like so:

    input_path
    └─── case1
        │─── *
    └─── case2
        │─── *
    └─── ...

Don't worry about preprocessing your dicoms, separating by scan type, or excluding non-cines. The pipeline should find which ones are cines and which ones aren't by checking key terms within the series descriptions. Check src/bivme/preprocessing/dicom/extract_cines.py for the list of key terms.

In [4]:
batch_ID = 'test' # This will serve as your output folder name. Example: 'test'
analyst_id = 'analyst1' # Example: 'analyst1'
input_path = '' # Path to the input DICOM folder
processed_path = '' # Path to the processed folder, where view predictions and segmentations will be stored. This will be created upon run time.
states_path = '' # Path to the states folder, where the logs and view predictions will be stored for reference. This will be created upon run time.
output_path = '' # Path to the output folder, where GP files will be stored. This will be created upon run time.
plotting_path = '' # Path to the plotting folder, where the HTML file of plotted guidepoints will be stored. This will be created upon run time.

# Target path: src/bivme/preprocessing/dicom/models
cwd = os.getcwd()
print('Current working directory:', cwd)
MODEL_DIR = os.path.join(cwd,'models')

Current working directory: c:\Users\jdil469\Code\biv-me\src\bivme\preprocessing\dicom


## Step 0.1: Choose case

In [8]:
case = '' # Enter the case ID. This should be a subfolder within the input_path. 

In [9]:
src = os.path.join(input_path)
dst = os.path.join(processed_path, batch_ID)
states = os.path.join(states_path, batch_ID)
output = os.path.join(output_path, batch_ID)
plotting = os.path.join(plotting_path, batch_ID)


case_src = os.path.join(src, case)
if not os.path.isdir(case_src):
    logger.error(f'Case {case} not found in source folder. Please check the case ID.')
    sys.exit()

os.makedirs(dst, exist_ok=True)
os.makedirs(states, exist_ok=True)
os.makedirs(output, exist_ok=True)
os.makedirs(plotting, exist_ok=True)

case_dst = os.path.join(dst, case)

if os.path.exists(case_dst):
    enter = input(f'Case {case} already processed. Do you want to overwrite? (y/n): ')
    if enter == 'y':
        shutil.rmtree(case_dst)
    else:
        print(f'Change the case ID or delete the existing folder {case_dst} before proceeding.')

In [10]:
# Create log file to record some details
states = os.path.join(states, case, analyst_id)
os.makedirs(states, exist_ok=True)

logger_id = logger.add(f'{states}/log_file_{datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.log', level=log_level, format=log_format,
    colorize=False, backtrace=True,
    diagnose=True)

logger.info(f'Processing case: {case}')

start_time = time.time()

Processing case: cardiohance_001


In [23]:
# Reload modules to ensure any changes are reflected
importlib.reload(sys.modules[extract_cines.__module__])
from extract_cines import extract_cines

## Step 0.2: Pre-preprocess dicoms
By default, the pipeline takes in only cardiac cine images in .dcm format. The pre-preprocessing reads in the raw dicoms and uses the series descriptions to infer which are cines and which aren't. The 'cine only' dicoms are saved to:

    processed_path   
    └───batch_ID
        └───processed-dicoms

In [ ]:
logger.info(f'Finding cines...')
extract_cines(case_src, case_dst, logger)

case_src = os.path.join(case_dst, 'processed-dicoms') # Update source directory

logger.success(f'Pre-preprocessing complete. Cines extracted to {case_src}.')

In [8]:
# Reload modules to ensure any changes are reflected
importlib.reload(sys.modules[select_views.__module__])
from select_views import select_views

## Step 1: View selection
View selection can be carried out in four main ways. 

The first option is the 'default' option. This involves the use of two models - one using only dicom metadata and one using only image data. Each model performs prediction separately and then predictions are combined for a final verdict. This should be the most robust option, but takes longer to complete.

The second option is the 'metadata-only' option. This only uses metadata. This model can struggle with subtle distinctions between similar views (e.g. between SAX and SAX-atria) but is excellent at distinguishing between general categories of view (e.g. SAX vs LAX).

The third option is the 'image-only' option. This only uses image data. This model is better all round than the metadata-only model, but can occasionally produce spurious predictions. 

The fourth option is the 'load' option. This loads view predictions from the states folder. This is useful if view predictions have already been made, avoiding the need to rerun prediction from scratch. It also allows for you to make and save manual corrections to the selected views. 

In [12]:
option = 'default' # Either 'default', 'metadata-only', 'image-only', or 'load'. 
# 'default' will combine dicom metadata and image data to select the correct view. 'metadata-only' will only use metadata. 'image-only' will only use the image data. 
# 'load' will load view predictions from the states folder, if view predictions have already been made.

slice_info_df, num_phases, slice_mapping = select_views(case, case_src, case_dst, MODEL_DIR, states, option, logger)

logger.success(f'View selection complete.')
logger.info(f'Number of phases: {num_phases}')

Data prepared for view prediction. 14 image series found.
Running view predictions...
No duplicate slice locations found.
Multiple series classed as 3ch.
Excluded series [15]
View predictions for cardiohance_001:
SAX-atria: 1 series
SAX: 5 series
4ch: 1 series
3ch: 1 series
Excluded: 1 series
2ch: 1 series
2ch-RT: 1 series
LVOT: 1 series
RVOT: 1 series
RVOT-T: 1 series
View selection complete.
Number of phases: 27


## Manually correct view predictions (optional)
No view prediction network is perfect. You can manually correct the view predictions if needed. The view predictions will be saved in:

    states_path   
    └───batch_ID
        └───case
            └───analyst_id
                |───view_predictions.csv

You can visually inspect which images have been classified as which view (or excluded) here:


    processed_path   
    └───batch_ID
        └───case
            └───view-classsification
                └───sorted

If you do correct the views, make run the cell block below with the variable 'option' set to 'load'. 

In [ ]:
option = 'load' # Either 'default', 'metadata-only', 'image-only', or 'load'. 
# 'default' will combine dicom metadata and image data to select the correct view. 'metadata-only' will only use metadata. 'image-only' will only use the image data. 
# 'load' will load view predictions from the states folder, if view predictions have already been made.

slice_info_df, num_phases, slice_mapping = select_views(case, case_src, case_dst, MODEL_DIR, states, option, logger)

logger.success(f'View selection complete.')
logger.info(f'Number of phases: {num_phases}')

In [13]:
# Reload modules to ensure any changes are reflected
importlib.reload(sys.modules[segment_views.__module__])
from segment_views import segment_views

## Step 2: Segmentation

In [14]:
## Segmentation
version = '3d' # '2d' or '3d' segmentation models. 3D is recommended for better efficiency and temporal coherence across frames.
seg_start_time = time.time()
segment_views(case, case_dst, MODEL_DIR, slice_info_df, version, logger)
seg_end_time = time.time()

logger.success(f'Segmentation complete. Time taken: {seg_end_time-seg_start_time} seconds ({version} version).')

Writing SAX images to nifti files...

Segmenting SAX images...

*** Making predictions for SAX images ***
There are 5 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 5 cases that I would like to predict

Predicting SAX_3d_10:
perform_everything_on_device: True


100%|██████████| 1/1 [00:03<00:00,  3.69s/it]


sending off prediction to background worker for resampling and export
done with SAX_3d_10

Predicting SAX_3d_11:
perform_everything_on_device: True


100%|██████████| 1/1 [00:00<00:00,  1.98it/s]


sending off prediction to background worker for resampling and export
done with SAX_3d_11

Predicting SAX_3d_12:
perform_everything_on_device: True


...

sending off prediction to background worker for resampling and export
done with 3ch_3d_14
Done with 3ch

Writing 4ch images to nifti files...

Segmenting 4ch images...

*** Making predictions for 4ch images ***
There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting 4ch_3d_13:
perform_everything_on_device: True


100%|██████████| 1/1 [00:02<00:00,  2.69s/it]


sending off prediction to background worker for resampling and export
done with 4ch_3d_13
Done with 4ch

Writing RVOT images to nifti files...

Segmenting RVOT images...

*** Making predictions for RVOT images ***
There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting RVOT_3d_19:
perform_everything_on_device: True


100%|██████████| 1/1 [00:02<00:00,  2.35s/it]


sending off prediction to background worker for resampling and export
done with RVOT_3d_19
Done with RVOT

Segmentation complete. Time taken: 95.30491971969604 seconds (3d version).


## Step 2.1: Correct for mismatching phases (conditional)
Often, LAX and SAX series do not have matching number of phases. In that case, we need to resample segmentations to have the same number of phases. We use the SAX series as the reference for the 'right' number of phases.

In [14]:
# Reload modules to ensure any changes are reflected
importlib.reload(sys.modules[correct_phase_mismatch.__module__])
from correct_phase_mismatch import correct_phase_mismatch

In [15]:
## Resample segmentations if phases are not consistent between SAX and LAX views
correct_phase_mismatch(case_dst, slice_info_df, num_phases, logger)

## Step 2.2: Review segmentations (optional)
Images and segmentations are stored here in nifti format (.nii.gz). 

    processed_path   
    └───batch_ID
        └───case


Hopefully it won't be necessary 99% of the time, but, if you wish, segmentations can be corrected here, and the rest of the code will incorporate those changes. 

By default, the contouring code (after segmentation) carries out some basic QC. All label types (except for the RV myocardium) have all but their largest components removed, so you shouldn't need to worry about fixing minor things such as removing extraneous label regions. If you find a problem with the exported contours or fitted models, it's more likely due to poor RV myo segmentation or poor segmentation around valve planes.   

3D Slicer or ITKSnap are good tools for correcting segmentations. You should only change the segmentations with '3d' in the name. These are the ones that get loaded later on for contouring & exporting. Both versions of segmentation model will output these '3d' segmentation files

In [12]:
# Reload modules to ensure any changes are reflected
importlib.reload(sys.modules[generate_contours.__module__])
from generate_contours import generate_contours

## Step 3: Generate contours

In [16]:
slice_dict = generate_contours(case, case_dst, slice_info_df, num_phases, version, logger)
logger.success(f'Guide points generated successfully.')

Generating contours for SAX slice 8...

Generating contours for SAX slice 9...

Generating contours for SAX slice 10...

Generating contours for SAX slice 11...

Generating contours for SAX slice 12...

Generating contours for 2ch slice 16...

Generating contours for 3ch slice 14...

Generating contours for 4ch slice 13...

Generating contours for RVOT slice 19...

Guide points generated.


In [17]:
# Reload to ensure any changes are reflected
importlib.reload(sys.modules[export_guidepoints.__module__])
from export_guidepoints import export_guidepoints

## Step 4: Export guidepoints

In [18]:
export_guidepoints(case, case_dst, output, slice_dict, slice_mapping)
logger.success(f'Guide points exported successfully.')
logger.success(f'Case {case} complete.')
logger.info(f'Total time taken: {time.time()-start_time} seconds.')

Export complete.
Case cardiohance_001 complete.
Total time taken: 102.80258440971375 seconds.


## Step 5 (Optional): Plot guidepoints
The code below will plot the guidepoints at each frame as html files, which can be viewed at:

    plotting_path   
    └───batch_ID
        └───case
            └───html

Here's a good place to find any issues before proceeding to model fitting. 

In [16]:
gp_dir = os.path.join(output, case)
generate_html(gp_dir, out_dir=plotting, gp_suffix='', si_suffix='', frames_to_fit=[], my_logger=logger, model_path=None)

logger.info(f'Guidepoints plotted and saved in {plotting}.')

2025-02-10 13:29:38.153 | INFO     | bivme.plotting.plot_guidepoints:generate_html:63 - case: cardiohance_045


Guidepoints plotted and saved in C:\Users\jdil469\bivme-data\fitting\plotting\cardiohance-lvh.
